In [1]:
from langchain_pinecone.vectorstores import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint
# from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
from pinecone import Pinecone
from dotenv import load_dotenv
import os

d:\_epistemology\truth_machine\il-legal\.venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
load_dotenv()
embeddings = HuggingFaceEmbeddings()
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
index = pc.Index("il-legal")

docsearch = PineconeVectorStore(index=index, embedding=embeddings)

In [5]:
repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
llm = HuggingFaceEndpoint(
    repo_id=repo_id, 
    model_kwargs={"huggingface_api_token":os.environ["HUGGINGFACEHUB_API_TOKEN"]},
    # model_kwargs={"temperature":0.5, "top_k":10},
    temperature=0.5,
    top_k=10,
    # huggingface_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"]
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\schel\.cache\huggingface\token
Login successful


In [6]:
prompt_template = """
    You are a trained bot to guide people about Illinois Crimnal Law Statutes and the Safe-T Act. You will answer user's query with your knowledge and the context provided. 
    If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
    Do not say thank you and tell you are an AI Assistant and be open about everything.
    Use the following pieces of context to answer the users question.
    Context: {context}
    Question: {question}
    Only return the helpful answer below and nothing else.
    Helpful answer:
    """

PROMPT = PromptTemplate(
    template=prompt_template, 
    input_variables=["context", "question"])

In [8]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [9]:
message_history = ChatMessageHistory()
memory = ConversationBufferMemory(
    memory_key="chat_history",
    output_key="answer",
    chat_memory=message_history,
    return_messages=True,
    )
retrieval_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(),  ##  pinecone.get_retriever(),
    # retriever=vectorstore.similarity_search(
    #     query="question",
    #     k=2,
    # ),
    # retriever=docsearch.as_retriever(
    #     search_kwargs={'k': 2}),
    return_source_documents=True,
    combine_docs_chain_kwargs={"prompt": PROMPT},
    memory= memory
    )